In [3]:
import pandas as pd
import json
import numpy as np

In [4]:
# 定义acq01-03,idcard原始文件列名
colums_name={
    'ACQ01': ['ugid', 'zuid', 'appid', 'utime', 'ctime', 'path', 'remote_addr', 'sessionId' ],
    'ACQ02': ['ugid' ,'zuid' ,'appid' ,'utime' ,'ctime' ,'ch' ,'eleId' ,'path' ,'remote_addr' ,'sessionid' ,'startTime' ,'endTime' ,'stayTime' ,'val'],
    'ACQ03': ['ugid' ,'zuid' ,'appid' ,'utime' ,'ctime' ,'ch' ,'clickTime' ,'eleId' ,'path' ,'remote_addr' ,'sessionId'],
    'IDCARD01': ['ugid' ,'zuid' ,'appid' ,'utime' ,'ctime' ,'address' ,'allTime' ,'birthday' ,'edit_address_time' ,'edit_birthday_time' 
             ,'edit_id_card_number_time' ,'edit_issuing_authority_time' ,'edit_name_time' ,'edit_period_of_validity_time' ,'edit_race_time' 
             ,'gender' ,'id_card_number' ,'issued_by' ,'name' ,'race' ,'remote_addr' ,'statistic_back_idcard_recognition_count' 
             ,'statistic_face_recognition_count' ,'statistic_front_idcard_recognition_count' ,'valid_date'],
    'CAL01': ['ugid', 'zuid', 'appid', 'mid', 'utime', 'ctime', 'count', 'date', 'type', 'duration', 'name', 'number', 'remote_addr'],
    'DEV01':['ugid', 'zuid', 'appid', 'mid', 'utime', 'ctime', 'imei', 'imsi', 'model', 'nativePhoneNum', 'providersName', 'remote_addr', 'swv', 'sysVersion', 'wifimac'],
    'CON01':['ugid', 'zuid', 'appid', 'mid', 'utime', 'ctime', 'remote_addr', 'name', 'phone']
}

In [5]:
data = []
with open('../data_mid/20171122_data.201706_201708.screen_all.log', 'r',  encoding="utf-8") as f:
    for line in f:
        data.append(line.split('\t'))

In [6]:
for item in data:
    ugid, zuid, appid, mid, utime = item[0:-1]
    if mid == 'CON01':
        print(item)
        break

['2505e3f2-1de0-421c-bb14-15a0383ea08c', '86a8e2d6a6644cd1b16c7aaaeb07ba77|093473230362268', 'jkand2', 'CON01', '', '[[1496663903.696,"27.187.88.48","强叔","13733368853"],[1496663903.768,"27.187.88.48","杨哥",""]]\n']


In [7]:

for item in data:
    #data中某些行存在多余的数据，8月第二列多了一个空字符串（去掉），12月第二列有乱码
    try:
        ugid, zuid, appid, mid, utime = item[0:-1]
    except:
        ugid = item[0]
        zuid,appid,mid,utime = item[2:-1]
        print(item)
        
    if len(str(appid)) > 20:
        print(item)

# 抛出各种异常
for item in data:
    try:
        ugid, zuid, appid, mid, unknwon = item[0:-1]
    except:
        ugid = item[0]
        zuid,appid,mid,unknwon = item[2:-1]
        print(ugid,zuid,appid,mid,unknwon)

In [8]:
processed_data = []
a = 0
for item in data:
    #data中某些行存在多余的数据，8月第二列多了一个空字符串（去掉），12月第二列有乱码
    try:
        ugid, zuid, appid, mid, utime = item[0:-1]
    except:
        ugid = item[0]
        zuid,appid,mid,utime = item[2:-1]
        continue
    
    if mid!='CON01':
        a+=1
    else:
        body = json.loads(item[-1])
        for sub_body in body:
            ctime, remote_addr, name, phone  = sub_body
            name = name.strip()
            name = name.replace('\r','')
            
            #有些ctime值为空，直接将其对应的sessionId用np.nan替换了
            try:
                sessionId = str(int(ctime*1000))
            except:
                sessionId = np.nan
            processed_data.append([ugid, zuid, appid, mid, utime, ctime, remote_addr, name, phone])

In [9]:
processed_data[0]

['2505e3f2-1de0-421c-bb14-15a0383ea08c',
 '86a8e2d6a6644cd1b16c7aaaeb07ba77|093473230362268',
 'jkand2',
 'CON01',
 '',
 1496663903.696,
 '27.187.88.48',
 '强叔',
 '13733368853']

In [10]:
tmp_pd = pd.DataFrame(processed_data)
tmp_pd.columns = ['ugid', 'zuid', 'appid', 'mid', 'utime', 'ctime', 'remote_addr', 'name', 'phone']

In [12]:
tmp_pd['name'] = tmp_pd['name'].replace('\r','')

In [13]:
tmp_pd

,ugid,zuid,appid,mid,utime,ctime,remote_addr,name,phone
0,2505e3f2-1de0-421c-bb14-15a0383ea08c,86a8e2d6a6644cd1b16c7aaaeb07ba77|093473230362268,jkand2,CON01,,1.496664e+09,27.187.88.48,强叔,13733368853
1,2505e3f2-1de0-421c-bb14-15a0383ea08c,86a8e2d6a6644cd1b16c7aaaeb07ba77|093473230362268,jkand2,CON01,,1.496664e+09,27.187.88.48,杨哥,
2,99a3e6ce-2d8c-4fdf-af51-c0736b6db101,66990ff74d18481aa7b9513ce94bd7ce|123456789001,jkios,CON01,,1.497248e+06,120.85.76.68,A,6228480086785614875
3,99a3e6ce-2d8c-4fdf-af51-c0736b6db101,66990ff74d18481aa7b9513ce94bd7ce|123456789001,jkios,CON01,,1.497248e+06,120.85.76.68,丹萍,132-4683-3187
4,99a3e6ce-2d8c-4fdf-af51-c0736b6db101,66990ff74d18481aa7b9513ce94bd7ce|123456789001,jkios,CON01,,1.497248e+06,120.85.76.68,亚桂,+8613580557597
5,99a3e6ce-2d8c-4fdf-af51-c0736b6db101,66990ff74d18481aa7b9513ce94bd7ce|123456789001,jkios,CON01,,1.497248e+06,120.85.76.68,付国,15817039711
6,99a3e6ce-2d8c-4fdf-af51-c0736b6db101,66990ff74d18481aa7b9513ce94bd7ce|123456789001,jkios,CON01,,1.497248e+06,120.85.76.68,付平,18980601201
7,99a3e6ce-2d8c-4fdf-af51-c0736b6db101,66990ff74d18481aa7b9513ce94bd7ce|123456789001,jkios,CON01,,1.497248e+06,120.85.76.68,余领导,138-0279-3488
8,99a3e6ce-2d8c-4fdf-af51-c0736b6db101,66990ff74d18481aa7b9513ce94bd7ce|123456789001,jkios,CON01,,1.497248e+06,120.85.76.68,关婉怡,139-2514-6353
9,99a3e6ce-2d8c-4fdf-af51-c0736b6db101,66990ff74d18481aa7b9513ce94bd7ce|123456789001,jkios,CON01,,1.497248e+06,120.85.76.68,冬梅,13697405055


In [15]:
tmp_pd.to_csv('../data_mid/20171122_data_con01.201706_201708.csv', header = True, index=False)